# PyTorch Cats vs Dogs Classifier

## import libraries

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import os
import import_ipynb
from SimpleCNN import SimpleCNN

# Import constants defined in another file
import Constants

## 1. Define function for loading data

In [3]:
def get_data_loaders():
    transform = {
        'train': transforms.Compose([
            transforms.Resize((IMAGE_SIZE, IMAGE_SIZE)),
            transforms.RandomHorizontalFlip(),
            transforms.ToTensor(),
        ]),
        'val': transforms.Compose([
            transforms.Resize((IMAGE_SIZE, IMAGE_SIZE)),
            transforms.ToTensor()
        ])
    }

    datasets_ = {
        x: datasets.ImageFolder(os.path.join(DATA_DIR, x), tranform=transform[x])
        for x in ['train', 'val']
    }

    loaders = {
        x: torch.utils.data.DataLoader(datasets_[x], batchSize=BATCH_SIZE, shuffle=True)
        for x in ['train', 'val']
    }

    return loaders, datasets_['train'].classes



## 2. Define the training function

In [4]:
def train_model(model, loaders, epoch=3):

    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(mode.parameters(), lr=0.001)

    model.to(DEVICE)
    model.train()

    for epoch in range(epochs):
        running_loss = 0.0
        for inputs, labels in loaders['train']:
            inputs, labels = inputs.to(DEVICE), labels.to(DEVICE)

            outputs = model(inputs)
            loss = criterion(outputs, labels)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            running_loss += loss.item()

            print(f'Epoch {epoch+1}, Loss: {running_loss:.4f}')

## 3. Define the function to evaluate the model

In [ ]:
def evaluate_model(model, loader):

    model.eval()
    correct = total = 0

    with torch.no_grad():
        for inputs, labels in loader:
            inputs, labels = inputs.to(DEVICE), labels.to(DEVICE)
            outputs = model(inputs)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    
    acc = 100 * correct / total
    print(f"Validation Accruacy: {acc:.2f}%")
    return acc

Validation Accuracy: 79.151661%


## Test the model on a single image

In [10]:
from PIL import Image

def predict_image(img_path):
    image = Image.open(img_path)
    image = tranform['val'](image).unsqueeze(0).to(device)
    output = model(image)
    _, predicted = torch.max(output, 1)
    print(f"Prediction: {class_names[predicted.item()]}")

predict_image("data/predict/dogs/set2-55.jpg")

Prediction: dogs


## Save the model

In [13]:
torch.save(model.state_dict(), 'models/catsvsDogsTorchModel.pth')